In [2]:
# One tone example

import spcm
from spcm import units
import numpy as np  
card : spcm.Card
with spcm.Card('/dev/spcm0') as card:
    card.card_mode(spcm.SPC_REP_STD_DDS)    # setup card for DDS
    # Setup the card
    channels = spcm.Channels(card)
    channels.enable(True)
    channels.output_load(50 * units.ohm)
    channels.amp(2.0 * units.V)
    card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work
    
    # Setup DDS
    dds = spcm.DDS(card, channels=channels)
    dds.reset()
   
    # Switch groups of cores to other channels
    if len(channels) == 2:
        # pass
        dds.cores_on_channel(1, 
                             spcm.SPCM_DDS_CORE8,  spcm.SPCM_DDS_CORE9,  spcm.SPCM_DDS_CORE10, spcm.SPCM_DDS_CORE11, # Flex core block 8 - 11
                             spcm.SPCM_DDS_CORE20) # Fixed core 20
        
    temp = 360*np.random.rand()*units.degrees  
    

    dds[0].amp(1000 * units.mV) 
    dds[0].freq(97.5 * units.MHz) 
    dds[0].phase(360*np.random.rand()*units.degrees)                        


    dds[8].amp(800 * units.mV)  
    dds[8].freq(97.5 * units.MHz) 
    dds[8].phase(360*np.random.rand()*units.degrees)                                                    
    
    dds.exec_at_trg()
    dds.write_to_card()

    # Start command including enable of trigger engine
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

    input("Press Enter to Exit")

In [2]:
#Tweezer example

import spcm
from spcm import units
import numpy as np  
card : spcm.Card
with spcm.Card('/dev/spcm0') as card:
    card.card_mode(spcm.SPC_REP_STD_DDS)    # setup card for DDS
    # Setup the card
    channels = spcm.Channels(card)
    channels.enable(True)
    channels.output_load(50 * units.ohm)
    channels.amp(1.0 * units.V)
    card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work
    
    # Setup DDS
    dds = spcm.DDS(card, channels=channels)
    dds.reset()
   
    # Switch groups of cores to other channels
    if len(channels) == 2:
        # pass
        dds.cores_on_channel(1, 
                             spcm.SPCM_DDS_CORE8,  spcm.SPCM_DDS_CORE9,  spcm.SPCM_DDS_CORE10, spcm.SPCM_DDS_CORE11, # Flex core block 8 - 11
                             spcm.SPCM_DDS_CORE20) # Fixed core 20
        
    temp = 360*np.random.rand()*units.degrees  
     
    list_x = [0,1,2,3,4,5,6,7,12,13,14,15,16,17,18,19]
    for i in range(len(list_x)):
        dds[list_x[i]].amp(30 * units.mV) 
        dds[list_x[i]].freq(92.5 * units.MHz + i*0.5*units.MHz) 
        dds[list_x[i]].phase(360*np.random.rand()*units.degrees)                        
    
    list_y = [8,9,10,11,20]
    for j in range (len(list_y)):   
        dds[list_y[j]].amp(30 * units.mV)  
        dds[list_y[j]].freq(96.5 * units.MHz + j*0.5*units.MHz) 
        dds[list_y[j]].phase(360*np.random.rand()*units.degrees)                                                    
    
    dds.exec_at_trg()
    dds.write_to_card()

    # Start command including enable of trigger engine
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

    input("Press Enter to Exit")

In [ ]:
import spcm,time
from spcm import units
import numpy as np
# import matplotlib.pyplot as plt


def shape_linear(x):
    return x

def shape_cosine(x):
    return 0.5-0.5*np.cos(np.pi*x)

def shape_square(x):
    return np.where(x < 0.5, 0, 1)

# record the beginning time of the program  
start_time = time.time()
print(f"Program beginning time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")

def calculate_slopes_for_given_shape(parameters,shape):
    frequency_range_Hz = parameters["endFreq_Hz"] - parameters["startFreq_Hz"]                      
    unit_slices_in = np.linspace(0, 1, num_segments+1, endpoint=True)
    unit_slices_out = shape(unit_slices_in)
    time_list = unit_slices_in*parameters["time_s"]         
    output_list = unit_slices_out*frequency_range_Hz+parameters["startFreq_Hz"]         
    t_diff = np.diff(time_list)
    y_diff = np.diff(output_list)
    return np.divide(y_diff, t_diff),time_list,output_list  


card : spcm.Card
with spcm.Card('/dev/spcm0') as card:       
    card.card_mode(spcm.SPC_REP_STD_DDS)    # setup card for DDS
    channels = spcm.Channels(card)
    channels.enable(True)
    channels.amp(1 * units.V)
    card.write_setup() # IMPORTANT! this turns on the card's system clock signals, that are required for DDS to work
    dds = spcm.DDS(card)    # Setup DDS
    dds.reset()
    dds.data_transfer_mode(spcm.SPCM_DDS_DTM_DMA)
    num_cores = len(dds)     # Start the DDS test
    first_init_freq_Hz  = 90 * units.MHz    # 20 Carriers from 90 to 110 MHz
    delta_init_freq_Hz  = 20*1/num_cores * units.MHz
    first_final_freq_Hz = 95  * units.MHz    # to 20 Carriers from 95 to 105 MHz
    delta_final_freq_Hz = 10*1/num_cores * units.MHz 
    
    # Ramp settings
    num_segments = 16
    total_time_s = 20.0 * units.s
    ramp_type = 'cosine' # 'cosine' or 'square' or '3rd-order'


    dds.freq_ramp_stepsize(1000)      # STEP 0 - Initialize frequencies
    dds.trg_timer(5.0 * units.s)
    dds.trg_src(spcm.SPCM_DDS_TRG_SRC_TIMER)
    for core in dds:
        core.amp(45 * units.percent / num_cores)
        core.freq(first_init_freq_Hz + int(core) * delta_init_freq_Hz)
    dds.exec_at_trg()
    dds.write_to_card()


    period_s = total_time_s / num_segments # seconds      # STEP 1 - Start the ramp
    dds.trg_timer(period_s) 
    
    parameters = []    # Define the parameters
    slopes = np.zeros((num_cores, num_segments))
    # plt.figure(figsize=(7,7))    # Show the results
    for core in dds:
        parameters = {
            "startFreq_Hz": first_init_freq_Hz + core.index * delta_init_freq_Hz, 
            "endFreq_Hz": first_final_freq_Hz + core.index * delta_final_freq_Hz, 
            "time_s": total_time_s, 
            }

        sl_core,time_list,output_list = calculate_slopes_for_given_shape(parameters,shape_cosine)
        slopes[core, :] = sl_core.to_base_units().magnitude

    #     plt.plot(time_list, output_list, 'ok')
    #     t_fine_s = np.linspace(time_list[0], time_list[1], 2, endpoint=True)
    #     for j, sl in enumerate(sl_core):
    #         plt.plot(t_fine_s + time_list[j], output_list[j] + sl*(t_fine_s), '--')

    # plt.show(block=False)

    # Do the slopes
    for j in range(num_segments):
        for core in dds:
            core.frequency_slope(slopes[core][j]) # Hz/s
        dds.exec_at_trg()

    # STEP 2 - Stop the ramp
    for core in dds:
        core.frequency_slope(0) # Hz/s
        core.freq(first_final_freq_Hz + core.index * delta_final_freq_Hz)
        core.phase(180*np.random.rand()*units.degrees)
    dds.exec_at_trg()
    dds.write_to_card()


    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"\nProgram ending time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time))}")
    print(f"Total time: {execution_time:.4f} s")
    # Start command including enable of trigger engine
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_FORCETRIGGER)

    input("Press Enter to Exit")

Program beginning time: 2025-09-30 17:55:50

Program ending time: 2025-09-30 17:55:50
Total time: 0.4773 秒
